In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.WARNING)

In [ ]:
import zcode.math as zmath

In [ ]:
sam = holo.sam.Semi_Analytic_Model(shape=(10, 11, 12))

In [ ]:
def dynamic_binary_number_eccentricity(sam, fobs_orb, eccen_init):
    """

    d^4 N / [dlog10(M) dq dz dln(X)    <===    d^3 n / dlog10(M) dq dz

    d^2 N / dz dln(f_r) = (dn/dz) * (dt/d ln f_r) * (dz/dt) * (dVc/dz)
                        = (dn/dz) * (f_r / [df_r/dt]) * 4 pi c D_c^2 (1+z)
                        = `dens`  *      `tau`        *   `cosmo_fact`

    """

    eccen_init = np.asarray(eccen_init) * np.ones(sam.shape)
    assert np.shape(eccen_)

    fobs_orb = np.asarray(fobs_orb)
    xsize = fobs_orb.size
    edges = sam.edges + [fobs_orb, ]

    # shape: (M, Q, Z)
    dens = sam.static_binary_density   # d3n/[dz dlog10(M) dq]  units: [Mpc^-3]

    # (Z,) comoving-distance in [Mpc]
    dc = cosmo.comoving_distance(sam.redz).to('Mpc').value

    # (Z,) this is `(dVc/dz) * (dz/dt)` in units of [Mpc^3/s]
    cosmo_fact = 4 * np.pi * (SPLC/MPC) * np.square(dc) * (1.0 + sam.redz)

    # (M, Q) calculate chirp-mass
    mchirp = utils.chirp_mass_mtmr(sam.mtot[:, np.newaxis], sam.mrat[np.newaxis, :])
    # (M, Q, 1, 1) make shape broadcastable for later calculations
    mchirp = mchirp[..., np.newaxis, np.newaxis]

    # (M*Q*Z,) 1D arrays of each total-mass, mass-ratio, and redshift
    mt, mr, rz = [gg.ravel() for gg in sam.grid]

    # Convert from observer-frame orbital freq, to rest-frame orbital freq
    # (X, M*Q*Z)
    frst_orb = fobs_orb[:, np.newaxis] * (1.0 + rz[np.newaxis, :])
    sa = utils.kepler_sepa_from_freq(mt[np.newaxis, :], frst_orb)



    # (X, M*Q*Z), hardening rate, negative values, units of [cm/sec]
    dadt = hard.dadt(mt[np.newaxis, :], mr[np.newaxis, :], sa)

    # Calculate `tau = dt/dlnf_r = f_r / (df_r/dt)`
    # dfdt is positive (increasing frequency)
    dfdt, frst_orb = utils.dfdt_from_dadt(dadt, sa, frst_orb=frst_orb)
    tau = frst_orb / dfdt


    # convert `tau` to the correct shape, note that moveaxis MUST happen _before_ reshape!
    # (X, M*Q*Z) ==> (M*Q*Z, X)
    tau = np.moveaxis(tau, 0, -1)
    # (M*Q*Z, X) ==> (M, Q, Z, X)
    tau = tau.reshape(dens.shape + (xsize,))

    # (M, Q, Z) units: [1/s] i.e. number per second
    dnum = dens * cosmo_fact
    # (M, Q, Z, X) units: [] unitless, i.e. number
    dnum = dnum[..., np.newaxis] * tau

    bads = ~np.isfinite(tau)
    if np.any(bads):
        log.warning(f"Found {utils.frac_str(bads)} invalid hardening timescales.  Setting to zero densities.")
        dnum[bads] = 0.0

    return edges, dnum

In [ ]:
NUM = 1000
mt = MSOL * (10.0 ** np.random.uniform(6, 10, NUM))
mr = (10.0 ** np.random.uniform(-2, 0, NUM))
rz = zmath.random_power([0, 1], +2, NUM) + 0.01

kale.dist1d(rz)
plt.show()